In [1]:
import data_preparation
import util_data
import util_config

import numpy as np

import os
import pickle
import csv

c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def print_data_info(prefix: str, data_arr: np.ndarray, label_arr: np.ndarray) -> str:
    result_str = f'<{prefix} Data Info.>\n'
    result_str += f'{prefix} X shape: {data_arr.shape}\n'
    result_str += f'{prefix} y shape: {label_arr.shape}\n'
    result_str += f'{(label_arr == 0.0).sum()}, {(label_arr == 1.0).sum()}, {(label_arr == 2.0).sum()}, {(label_arr == 3.0).sum()}\n'
    return result_str

In [3]:
import torch
from torch import nn
from torch import optim
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


class TargetModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, gpu=False, dropout=0.3):
        super(TargetModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.max_seq_len = max_seq_len
        self.gpu = gpu

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim)
        self.dense = nn.Linear(hidden_dim, hidden_dim)   # hidden_dim
        self.dropout = nn.Dropout(p=dropout)
        self.classifier = nn.Linear(hidden_dim, 1)       # hidden_dim

        if self.gpu:
            self.cuda()

    def init_hidden(self, batch_size):
        h = torch.autograd.Variable(torch.zeros(1, batch_size, self.hidden_dim))

        if self.gpu:
            return h.cuda()
        else:
            return h

    def forward(self, inp, hidden, cell_s, sig):
        # input dim                                                # batch_size x seq_len
        emb = self.embeddings(inp)                               # batch_size x seq_len x embedding_dim
        emb = emb.permute(1, 0, 2)                                 # seq_len x batch_size x embedding_dim

        out, (hidden, cell_s) = self.lstm1(emb, (hidden, cell_s))      # (seq_len, B, hidden_dim), (1, B, hidden_dim), (1, B, hidden_dim)
        out, (_, _) = self.lstm2(out, (hidden, cell_s))
        out = out[-1]
        out = self.dense(out)
        out = self.dropout(out)
        out = self.classifier(out)
        if sig:
            out = torch.sigmoid(out)
        return out

    def batchClassify(self, inp, sig):
        h = self.init_hidden(inp.size()[0])
        c = self.init_hidden(inp.size()[0])
        out = self.forward(inp, h, c, sig)
        return out.view(-1)
    
    def do_train(self, x_train, x_val, y_train, y_val, lr=0.0001, batch_size=64, epochs=10, weight_decay=0.02):
        self.train()

        x_train = torch.Tensor(x_train).type(torch.LongTensor)
        x_val = torch.Tensor(x_val).type(torch.LongTensor)
        y_train = torch.Tensor(y_train)
        y_val = torch.Tensor(y_val)
        if self.gpu:
            x_train = x_train.cuda()
            x_val = x_val.cuda()
            y_train = y_train.cuda()
            y_val = y_val.cuda()

        optimizer = optim.RMSprop(self.parameters(), lr=lr, weight_decay=weight_decay)
        loss_fn = nn.BCELoss()

        for epoch in range(epochs):
            print(f'Epoch: {epoch}')
            total_loss = 0
            total_acc = 0
            for idx in range(int(x_train.shape[0] / batch_size) + 1):
                inp = x_train[idx * batch_size: (idx+1) * batch_size]
                label = y_train[idx * batch_size: (idx+1) * batch_size]

                optimizer.zero_grad()
                out = self.batchClassify(inp, sig=True)
                loss = loss_fn(out, label)
                loss.backward()
                optimizer.step()

                total_loss += loss.data.item()
                total_acc += torch.sum((out > 0.5) == (label > 0.5)).data.item()
            total_acc /= x_train.shape[0]

            val_loss = 0
            val_acc = 0
            val_batch_size = 1024
            for idx in range(int(x_val.shape[0] / val_batch_size) + 1):
                inp = x_val[idx * val_batch_size: (idx+1) * val_batch_size]
                label = y_val[idx * val_batch_size: (idx+1) * val_batch_size]

                val_out = self.batchClassify(inp, sig=True)
                val_loss += loss_fn(val_out, label).data.item()
                val_acc += torch.sum((val_out > 0.5) == (label > 0.5)).data.item()
            val_acc /= x_val.shape[0]
            val_loss /= int(x_val.shape[0] / val_batch_size) + 1

            print(f'Loss sum: {total_loss}, Loss avg: {total_loss / (int(x_train.shape[0] / batch_size) + 1)}, Acc: {total_acc}')
            print(f'Val loss: {val_loss}, Val acc: {val_acc}\n')
    
    def do_evaluate(self, x_test, y_test, additional_info='', only_accuracy=False):
        self.eval()
        x_test = torch.Tensor(x_test).type(torch.LongTensor)
        if self.gpu:
            x_test = x_test.cuda()
        
        inf_batch_size = 256
        y_test_predicted = []
        for idx in range(int(x_test.shape[0] / inf_batch_size) + 1):
            inp = x_test[idx * inf_batch_size: (idx+1) * inf_batch_size]

            prediction = self.batchClassify(inp, sig=True).cpu().detach().numpy().round()
            y_test_predicted.append(prediction)
        y_test_predicted = np.hstack(y_test_predicted)

        desc = f'========={additional_info}=========\n'
        acc = f'{accuracy_score(y_test, y_test_predicted):.02%}'
        desc += f'Accuracy: {acc}' + '\n'
        result = {}

        if not only_accuracy:
            desc += str(confusion_matrix(y_test, y_test_predicted)) + '\n'
            result = classification_report(y_test, y_test_predicted, 
                                           labels=[0.0, 1.0], target_names=['attack-free', 'attack'], digits=4)
            desc += str(result) + '\n\n'

        return desc, result, acc

In [12]:
def train_evaluate_target_model(seq_len: int, exp_type: str, aug_label: str, gan_epoch_idx: int = -1,
                                train_epoch: int = 1, label_info_list: list = None, exp_id: str = None):
    util_config.CONFIG.reload_param_config()
    util_config.CONFIG.reload_path_config()
    param_config = util_config.CONFIG.get_param_config()
    path_config = util_config.CONFIG.get_path_config()

    num_of_msg = len(util_data.INT_TO_MSG_ID_CONVERTER)

    if param_config['is_binary']:
        num_of_labels = len(param_config['binary_label_list'])
    else:
        num_of_labels = len(param_config['full_label_list'])

    if label_info_list is None:
        label_info_list = param_config['label_info_list']
        print(f'Label Info List is automatically set to JSON.')
    if exp_id is None:
        exp_id = param_config['exp_id']
        print(f'Exp ID is automatically set to JSON.')

    # Load data
    train_data, train_label, test_data, test_label, test_data_2, test_label_2, new_inf_dict \
        = data_preparation.load_data(seq_len, label_info_list, exp_type, exp_id, gan_epoch_idx)

    # Logging info.
    info = f'Sequence Length: {seq_len}\n'
    info += f'# of messages: {num_of_msg}\n'
    info += f'# of labels: {num_of_labels}\n'
    info += f'Experiment type: {exp_type}\n'
    if exp_type in ['seqgan_model', 'maskgan_model', 'rankgan_model', 'stepgan_model', 'leakgan_model', 'gan_aug_gen_only']:
        info += f'GAN epoch index: {gan_epoch_idx}\n'
        
    info += '\n'
    info += print_data_info('train', train_data, train_label)
    info += '\n'
    info += print_data_info('test', test_data, test_label)
    info += '\n'
    info += print_data_info('test2', test_data_2, test_label_2)
    info += '\n'
    print(info)

    # Shuffling
    s = np.arange(train_data.shape[0])
    np.random.shuffle(s)
    train_data, train_label = train_data[s], train_label[s]

    embedding_dim = 32  # 32
    hidden_dim = 128     # 128
    model = TargetModel(embedding_dim, hidden_dim, num_of_msg, 128, gpu=True)

    # 저장 디렉토리 가져오기
    result_data_dir_path = os.path.join(path_config['result_data_dir_path'], f'{exp_type}_{aug_label}')
    if not os.path.exists(result_data_dir_path):
        os.makedirs(result_data_dir_path, exist_ok=True)

    # Pickle 파일로부터 dictionary 초기화
    result_info_file_path = os.path.join(result_data_dir_path, 'result_info.pickle')
    if os.path.exists(result_info_file_path):
        with open(result_info_file_path, 'rb') as info_file:
            result_info = pickle.load(info_file)
    else:
        result_info = []

    if exp_type == 'before_aug_none':
        weight_decay = 0.0
    else:
        weight_decay = 0.02
    
    batch_size = param_config['target_model']['batch_size']
    lr = param_config['target_model']['lr']
    model.do_train(train_data, test_data, train_label, test_label, lr=lr, batch_size=batch_size, epochs=train_epoch, weight_decay=weight_decay)

    old_new_performance = f'{len(result_info)}' + '\n'
    desc, _, _ = model.do_evaluate(test_data, test_label, additional_info='old_dataset')
    old_new_performance += desc
    desc, _, _ = model.do_evaluate(test_data_2, test_label_2, additional_info='new_dataset')
    old_new_performance += desc

    new_inf_key_list = list(new_inf_dict.keys())
    new_dict = {'iter': len(result_info)}
    for additional_label in new_inf_key_list:
        _, _, acc = model.do_evaluate(new_inf_dict[additional_label]['data'], new_inf_dict[additional_label]['label'], 
                                           additional_info=additional_label, only_accuracy=True)
        new_dict[additional_label] = acc

    result_info.append({'old_new_performance': old_new_performance, 'new_acc': new_dict})

    # export
    with open(result_info_file_path, 'wb') as info_file:
        pickle.dump(result_info, info_file)
    with open(os.path.join(result_data_dir_path, 'old_new_performance.txt'), 'a') as txt_file:
        txt_file.write(old_new_performance)
    with open(os.path.join(result_data_dir_path, 'new_accuracy.csv'), 'a', newline='\n') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=['iter'] + new_inf_key_list)
        writer.writeheader()
        writer.writerow(new_dict)

    return

In [13]:
def run_model(augmentation_type: str, exp_id: str, _gan_epoch_idx: int, n_iter: int):
    label_info_list_list = [
        [["normal", 0, True, True],
        ["heartbeat", 1, True, False],
        ["ping", 1, True, False],
        ["request", 1, True, False]],

        [["normal", 0, True, False],
        ["heartbeat", 1, True, True],
        ["ping", 1, True, False],
        ["request", 1, True, False]],

        [["normal", 0, True, True],
        ["heartbeat", 1, True, True],
        ["ping", 1, True, True],
        ["request", 1, True, True]]
    ]
    aug_labels = ['normal', 'heartbeat', 'all']

    seq_len = 128
    train_epoch = 5

    for label_info_list, aug_label in zip(label_info_list_list, aug_labels):
        for _ in range(n_iter):
            train_evaluate_target_model(seq_len, augmentation_type, aug_label, _gan_epoch_idx, train_epoch, 
                                        label_info_list=label_info_list, exp_id=exp_id)
        
        if augmentation_type in ['before_aug_none', 'before_aug_wr']:
            break

# None

In [14]:
run_model('before_aug_none', None, -1, 15)

Exp ID is automatically set to JSON.
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: before_aug_none

<train Data Info.>
train X shape: (628045, 128)
train y shape: (628045,)
158730, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 48.83573952689767, Loss avg: 0.15907407012018784, Acc: 0.9498650574401516
Val loss: 0.06598979098778693, Val acc: 0.9855252947508073

Epoch: 1
Loss sum: 15.006471353583038, Loss avg: 0.04888101418105224, Acc: 0.9880327046628824
Val loss: 0.02350216167556288, Val acc: 0.9950937445242685

Epoch: 2
Loss sum: 7.816562667256221, Loss avg: 0.02546111617998769, Acc: 0.9932616293418465
Val loss: 0.027309252719040705, Val acc: 0.9883288693083682

Epoch: 3
Loss sum: 3.239990808739094, Loss avg: 0.010553715989378156, Acc: 0.9963951627669991
Val loss: 0.0033264458690572696, Val ac

In [14]:
c

{'attack-free': {'precision': 0.02531645569620253,
  'recall': 2.1531770126822127e-05,
  'f1-score': 4.302694562469747e-05,
  'support': 92886},
 'attack': {'precision': 0.7299514759283972,
  'recall': 0.9996934054295111,
  'f1-score': 0.8437890166174031,
  'support': 251146},
 'accuracy': 0.7297896707283044,
 'macro avg': {'precision': 0.37763396581229985,
  'recall': 0.499857468599819,
  'f1-score': 0.42191602178151394,
  'support': 344032},
 'weighted avg': {'precision': 0.5397054276268217,
  'recall': 0.7297896707283044,
  'f1-score': 0.61598407406365,
  'support': 344032}}

# WR

In [7]:
run_model('before_aug_wr', None, -1, 15)

Exp ID is automatically set to JSON.
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: before_aug_wr

<train Data Info.>
train X shape: (628045, 128)
train y shape: (628045,)
158730, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 80.0777686201036, Loss avg: 0.2608396371990345, Acc: 0.8839143691932903
Val loss: 0.2654965694572563, Val acc: 0.9177826729079577

Epoch: 1
Loss sum: 16.248319935053587, Loss avg: 0.05292612356694979, Acc: 0.9857844581200391
Val loss: 0.023984900533346235, Val acc: 0.9947620716413427

Epoch: 2
Loss sum: 11.401285317726433, Loss avg: 0.03713773719129131, Acc: 0.9911487234195002
Val loss: 0.021269058240696217, Val acc: 0.9957195424165811

Epoch: 3
Loss sum: 13.990200228057802, Loss avg: 0.045570684781947236, Acc: 0.9871235341416619
Val loss: 0.030175375

# Oversampling

In [8]:
run_model('oversampling', None, -1, 10)

Exp ID is automatically set to JSON.
Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/oversampling
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: oversampling

<train Data Info.>
train X shape: (659791, 128)
train y shape: (659791,)
190476, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 87.55477560311556, Loss avg: 0.27106741672791196, Acc: 0.8802833018334594
Val loss: 0.053687799647806365, Val acc: 0.9886793161280633

Epoch: 1
Loss sum: 16.779004212468863, Loss avg: 0.05194738146275189, Acc: 0.9871383513870301
Val loss: 0.030743525677900407, Val acc: 0.9957383163533505

Epoch: 2
Loss sum: 13.940646483562887, Loss avg: 0.04315989623394083, Acc: 0.9896452058303311
Val loss: 0.03199934236301928, Val acc: 0.9944742046108789

Epoch: 

# Undersampling

In [9]:
run_model('undersampling', None, -1, 10)

Exp ID is automatically set to JSON.
Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/undersampling
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: undersampling

<train Data Info.>
train X shape: (501061, 128)
train y shape: (501061,)
31746, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 43.66795625537634, Loss avg: 0.1782365561443932, Acc: 0.930876679685707
Val loss: 0.49826816386976247, Val acc: 0.75

Epoch: 1
Loss sum: 36.89217837154865, Loss avg: 0.15058031988387205, Acc: 0.934969993673425
Val loss: 0.3602623167865951, Val acc: 0.75

Epoch: 2
Loss sum: 35.32683612406254, Loss avg: 0.14419116785331648, Acc: 0.9365346734229965
Val loss: 0.3518174192823683, Val acc: 0.75

Epoch: 3
Loss sum: 32.33716340363026, Loss avg: 0.1319884

c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaco

Exp ID is automatically set to JSON.
Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/undersampling
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: undersampling

<train Data Info.>
train X shape: (501061, 128)
train y shape: (501061,)
31746, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 43.44017015397549, Loss avg: 0.177306816955002, Acc: 0.9314414811769425
Val loss: 0.40975489790082736, Val acc: 0.75

Epoch: 1
Loss sum: 36.42034712433815, Loss avg: 0.14865447805852305, Acc: 0.9348462562442497
Val loss: 0.35834900372525214, Val acc: 0.75

Epoch: 2
Loss sum: 33.116417571902275, Loss avg: 0.13516905131388685, Acc: 0.934638696685633
Val loss: 0.3186848521110999, Val acc: 0.75

Epoch: 3
Loss sum: 32.037735283374786, Loss avg: 0.1307

c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Exp ID is automatically set to JSON.
Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/undersampling
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: undersampling

<train Data Info.>
train X shape: (501061, 128)
train y shape: (501061,)
31746, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 43.35030497610569, Loss avg: 0.17694002031063547, Acc: 0.9344650651317904
Val loss: 0.38464163056912887, Val acc: 0.75

Epoch: 1
Loss sum: 35.394635550677776, Loss avg: 0.14446790020684808, Acc: 0.9361874103153109
Val loss: 0.33496159803312103, Val acc: 0.75

Epoch: 2
Loss sum: 33.548452228307724, Loss avg: 0.1369324580747254, Acc: 0.9366204913174244
Val loss: 0.32087246005595393, Val acc: 0.75

Epoch: 3
Loss sum: 33.50115954130888, Loss avg: 0.1

c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaco

Exp ID is automatically set to JSON.
Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/undersampling
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: undersampling

<train Data Info.>
train X shape: (501061, 128)
train y shape: (501061,)
31746, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 42.716112710535526, Loss avg: 0.1743514804511654, Acc: 0.9358381514426387
Val loss: 0.3659412654704017, Val acc: 0.75

Epoch: 1
Loss sum: 35.850476518273354, Loss avg: 0.1463284755847892, Acc: 0.936616499787451
Val loss: 0.3133903269294151, Val acc: 0.75

Epoch: 2
Loss sum: 32.575599014759064, Loss avg: 0.13296162863166966, Acc: 0.9369677544251099
Val loss: 0.3118513824691059, Val acc: 0.7682420085609152

Epoch: 3
Loss sum: 18.730762861669064, Lo

c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaconda3\envs\uavproj\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\user\anaco

Exp ID is automatically set to JSON.
Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/undersampling
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: undersampling

<train Data Info.>
train X shape: (501061, 128)
train y shape: (501061,)
31746, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 43.82572565972805, Loss avg: 0.1788805128968492, Acc: 0.9347544510548615
Val loss: 0.45069926610263955, Val acc: 0.75

Epoch: 1
Loss sum: 37.5547127276659, Loss avg: 0.1532845417455751, Acc: 0.9357483420182373
Val loss: 0.30771144090819463, Val acc: 0.75

Epoch: 2
Loss sum: 31.94966220110655, Loss avg: 0.13040678449431245, Acc: 0.9376562933455208
Val loss: 0.2525192380298499, Val acc: 0.7831234824401111

Epoch: 3
Loss sum: 17.369136828929186, Los

# Random

In [10]:
run_model('random', None, -1, 10)

Exp ID is automatically set to JSON.
Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/random
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: random

<train Data Info.>
train X shape: (659791, 128)
train y shape: (659791,)
190476, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 86.65264388918877, Loss avg: 0.2682744392854141, Acc: 0.8811032584560868
Val loss: 0.15504177941222955, Val acc: 0.9474079451300408

Epoch: 1
Loss sum: 48.579555839300156, Loss avg: 0.15040110166965992, Acc: 0.9476000733565629
Val loss: 0.05069507509736926, Val acc: 0.9881911937720593

Epoch: 2
Loss sum: 22.056268533691764, Loss avg: 0.06828566109502095, Acc: 0.9793752870227087
Val loss: 0.034908530103624054, Val acc: 0.9886229943177551

Epoch: 3
Loss sum: 19

# Noise

In [11]:
run_model('noise', None, -1, 10)

Exp ID is automatically set to JSON.
Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/noise
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: noise

<train Data Info.>
train X shape: (659791, 128)
train y shape: (659791,)
190476, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 73.41516052186489, Loss avg: 0.22729151864354455, Acc: 0.9037286049673305
Val loss: 0.04257021465629434, Val acc: 0.9921775263460912

Epoch: 1
Loss sum: 28.347714841365814, Loss avg: 0.08776382303828426, Acc: 0.9743661250305021
Val loss: 0.026870685540566777, Val acc: 0.9951187764399609

Epoch: 2
Loss sum: 21.14126849733293, Loss avg: 0.0654528436449936, Acc: 0.9805665733542894
Val loss: 0.016588640950952367, Val acc: 0.9983541515432176

Epoch: 3
Loss sum: 20.6

# SeqGAN

In [7]:
run_model('seqgan_model', '(3)', 499, 20)

Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/Unrolled-0_wasserstein-False(3)
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: seqgan_model
GAN epoch index: 499

<train Data Info.>
train X shape: (678045, 128)
train y shape: (678045,)
208730, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 101.15559501945972, Loss avg: 0.3046855271670474, Acc: 0.8512517605763629
Val loss: 0.25495015473882104, Val acc: 0.8789331397531853

Epoch: 1
Loss sum: 74.22667648643255, Loss avg: 0.2235743267663631, Acc: 0.9053469902440103
Val loss: 0.10893490408683658, Val acc: 0.9748304087711832

Epoch: 2
Loss sum: 28.09863836877048, Loss avg: 0.08463445291798337, Acc: 0.9763761992198158
Val loss: 0.164585715687721, Val acc: 0.9240031039575459

Epoch: 3
Lo

# MaskGAN

In [8]:
run_model('maskgan_model', '(mask1)', 19, 20)

Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/Unrolled-0_wasserstein-False(mask1)
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: maskgan_model
GAN epoch index: 19

<train Data Info.>
train X shape: (678219, 128)
train y shape: (678219,)
208904, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 62.51127476617694, Loss avg: 0.18828697218727994, Acc: 0.9256700269382013
Val loss: 0.05921653743275696, Val acc: 0.9770707652256627

Epoch: 1
Loss sum: 14.650519413873553, Loss avg: 0.04412807052371552, Acc: 0.9899545721957067
Val loss: 0.053594044457148216, Val acc: 0.9901624571328443

Epoch: 2
Loss sum: 25.395374454557896, Loss avg: 0.07649209173059607, Acc: 0.9763822600074608
Val loss: 0.0431420874535729, Val acc: 0.9864765075471226

Ep

# RankGAN

In [9]:
run_model('rankgan_model', '(rank2)', 19, 20)

Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/Unrolled-0_wasserstein-False(rank2)
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: rankgan_model
GAN epoch index: 19

<train Data Info.>
train X shape: (673281, 128)
train y shape: (673281,)
203966, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 94.82606711983681, Loss avg: 0.2882251280238201, Acc: 0.8665475484975812
Val loss: 0.24089288103815382, Val acc: 0.9001414303236627

Epoch: 1
Loss sum: 27.46684935875237, Loss avg: 0.08348586431231723, Acc: 0.9741905682768414
Val loss: 0.08739692546800966, Val acc: 0.9605184109739918

Epoch: 2
Loss sum: 16.332906085066497, Loss avg: 0.04964409144397112, Acc: 0.9879099514170161
Val loss: 0.03862798621189917, Val acc: 0.9906505794888483

Epoc

# StepGAN

In [10]:
run_model('stepgan_model', '(step1)', 19, 20)

Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/Unrolled-0_wasserstein-False(step1)
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: stepgan_model
GAN epoch index: 19

<train Data Info.>
train X shape: (678221, 128)
train y shape: (678221,)
208906, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 107.33282519876957, Loss avg: 0.3232916421649686, Acc: 0.8588837561797703
Val loss: 0.35869688303417463, Val acc: 0.8438383939522892

Epoch: 1
Loss sum: 69.74741460382938, Loss avg: 0.21008257410791983, Acc: 0.9318806111872089
Val loss: 0.06564423817975126, Val acc: 0.992753260407019

Epoch: 2
Loss sum: 56.1722454726696, Loss avg: 0.1691935104598482, Acc: 0.9528531260459349
Val loss: 0.07422974176562516, Val acc: 0.9866141830834314

Epoch: 

# LeakGAN

In [11]:
run_model('leakgan_model', '(leak1)', 0, 20)

Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/Unrolled-0_wasserstein-False(leak1)
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: leakgan_model
GAN epoch index: 0

<train Data Info.>
train X shape: (678045, 128)
train y shape: (678045,)
208730, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 104.46754679083824, Loss avg: 0.314661285514573, Acc: 0.8419426439248132
Val loss: 0.21178636115910426, Val acc: 0.8978885579113369

Epoch: 1
Loss sum: 67.44460032880306, Loss avg: 0.20314638653253936, Acc: 0.9157327316033597
Val loss: 0.10646176100322965, Val acc: 0.9598300332924479

Epoch: 2
Loss sum: 29.1799109056592, Loss avg: 0.08789129790861204, Acc: 0.9720991969559543
Val loss: 0.24045411917932663, Val acc: 0.9257052742246364

Epoch: 

In [10]:
run_model('leakgan_model', '(leak1)', 0, 5)

Augmented data directory: D:/tasks/Projects/2022/무인이동체/논문작업/hitl\data2/augmented_data/1.0_128/Unrolled-0_wasserstein-False(leak1)
Shortage Ratio: 1.0
Sequence Length: 128
# of messages: 29
# of labels: 2
Experiment type: leakgan_model
GAN epoch index: 0

<train Data Info.>
train X shape: (678045, 128)
train y shape: (678045,)
208730, 469315, 0, 0

<test Data Info.>
test X shape: (159796, 128)
test y shape: (159796,)
39949, 119847, 0, 0

<test2 Data Info.>
test2 X shape: (344032, 128)
test2 y shape: (344032,)
92886, 251146, 0, 0


Epoch: 0
Loss sum: 91.5228131711483, Loss avg: 0.27567112400948285, Acc: 0.8789387135072156
Val loss: 0.1853876902965034, Val acc: 0.9277954391849609

Epoch: 1
Loss sum: 34.34148187376559, Loss avg: 0.10343819841495659, Acc: 0.9703574246547058
Val loss: 0.04104353137504144, Val acc: 0.9928095822173271

Epoch: 2
Loss sum: 29.42362029477954, Loss avg: 0.08862536233367331, Acc: 0.9749780619280431
Val loss: 0.04980492475590888, Val acc: 0.9962890184985856

Epoch: 